# Laboratorio 8
### Oscar Fernando López Barrios
### Carné 20679

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/MyDrive/SDS/malimg_paper_dataset_imgs"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install adversarial-robustness-toolbox

### Ataque 1: Fast Gradient Method

El modelo de Fast Gradient Method es una técnica fundamental en el campo del aprendizaje automático, especialmente cuando se trata de evaluar la robustez de los modelos en entornos adversarios.

Este método nos permite evaluar qué tan resistente es nuestro modelo ante estos ataques manipulativos. Funciona generando pequeñas perturbaciones en los datos de entrada, imperceptibles para los humanos, pero que pueden causar que el modelo clasifique incorrectamente la imagen.

Esta capacidad para generar imágenes "adversarias" nos ayuda a entender las debilidades de nuestro modelo y nos permite fortalecerlo. Al evaluar cómo responde nuestro modelo a estas perturbaciones, podemos identificar áreas que necesitan mejorar y desarrollar estrategias para hacer que el modelo sea más robusto frente a tales ataques.

En resumen, el Fast Gradient Method es una herramienta esencial para garantizar que nuestros modelos de aprendizaje automático sean confiables y seguros en entornos donde la seguridad es primordial, como en sistemas de reconocimiento de imágenes en aplicaciones médicas o de seguridad.

In [ ]:
import numpy as np
from tensorflow.keras.models import load_model
from art.attacks.evasion import FastGradientMethod
from art.estimators.classification import KerasClassifier
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

tf.compat.v1.disable_eager_execution()

physical_devices = tf.config.list_physical_devices('GPU')
print("GPUs:", physical_devices)

# Check if TensorFlow is using the GPU
if physical_devices:
    print("TensorFlow is using the GPU")
else:
    print("TensorFlow is not using the GPU")

GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
TensorFlow is using the GPU


In [ ]:
model = load_model('modelo_lab_7.h5')

classifier = KerasClassifier(model=model, clip_values=(0, 1))

attack = FastGradientMethod(estimator=classifier, eps=0.1)

data = ImageDataGenerator(rescale=1./255)
test_generator = data.flow_from_directory(path,
    target_size=(64, 64),
    batch_size=1,
    class_mode='categorical',
    shuffle=False)

x_test_adv = []
y_true = []
for i in range(len(test_generator)):
    x, y = test_generator[i]
    x_adv = attack.generate(x=x)
    x_test_adv.append(x_adv[0])
    y_true.append(y[0])

y_pred_adv = classifier.predict(np.array(x_test_adv))

y_pred_adv_indices = np.argmax(y_pred_adv, axis=1)
y_true_indices = np.argmax(y_true, axis=1)

accuracy = np.mean(y_pred_adv_indices == y_true_indices)
print("Accuracy:", accuracy)

precision = np.sum(y_pred_adv_indices == y_true_indices) / len(y_true_indices)
print("Precision:", precision)

Found 9339 images belonging to 25 classes.
Accuracy: 0.10429382160830925
Precision: 0.10429382160830925


### Ataque 2: DeepFool
DeepFool es una excelente herramienta para evaluar cuán robusto es nuestro modelo de aprendizaje automático frente a pequeñas perturbaciones en los datos de entrada. Esto es clave, especialmente en aplicaciones donde incluso pequeños cambios en los datos pueden tener un gran impacto en las decisiones del modelo.

Mediante el análisis de cómo nuestro modelo responde a las perturbaciones generadas por DeepFool, podemos obtener una comprensión más profunda de sus decisiones y límites. Esta comprensión nos permite identificar áreas de mejora y realizar ajustes para fortalecer nuestro modelo.

En conclusión, DeepFool es una herramienta valiosa que nos permite evaluar la robustez y seguridad de nuestros modelos de aprendizaje automático, mejorar su capacidad de generalización y comprensión, y garantizar su eficacia en aplicaciones críticas.

In [ ]:
from art.estimators.classification import KerasClassifier
from art.attacks.evasion import DeepFool

model_logits = tf.keras.Model(inputs=modelo.input, outputs=modelo.layers[-2].output)

classifier = KerasClassifier(model=model_logits, use_logits=True, clip_values=(0, 1))

attack = DeepFool(classifier=classifier)

x_test, y_test = next(test)
x_test_adv = attack.generate(x=x_test)

y_pred_adv = classifier.predict(x_test_adv)

y_pred_adv_indices = np.argmax(y_pred_adv, axis=1)
y_true_indices = np.argmax(y_test, axis=1)

accuracy = np.mean(y_pred_adv_indices == y_true_indices)
print("Accuracy:", accuracy)

precision = np.sum(y_pred_adv_indices == y_true_indices) / len(y_true_indices)
print("Precision:", precision)

DeepFool:   0%|          | 0/1 [00:00<?, ?it/s]

Accuracy: 0.0
Precision: 0.0


In [ ]:
from art.defences.trainer import AdversarialTrainer

# Crear un clasificador de ART basado en el modelo de Keras
classifier = KerasClassifier(model=model, clip_values=(0, 1))

# Configurar los ataques FGSM y DeepFool
fgsm_attack = FastGradientMethod(estimator=classifier, eps=0.1)
model_logits = tf.keras.Model(inputs=model.input, outputs=model.layers[-2].output)
logits_classifier = KerasClassifier(model=model_logits, use_logits=True, clip_values=(0, 1))
deepfool_attack = DeepFool(classifier=logits_classifier)

data = ImageDataGenerator(rescale=1./255)
test_generator = data.flow_from_directory(path,
    target_size=(64, 64),
    batch_size=1,
    class_mode='categorical',
    shuffle=False)

x_test = []
y_test = []
for i in range(len(test_generator)):
    x, y = test_generator.next()
    x_test.append(x[0])
    y_test.append(y[0])

x_test = np.array(x_test)
y_test = np.array(y_test)

# Implementar la defensa mediante entrenamiento adversarial
adv_trainer = AdversarialTrainer(classifier, attacks=[fgsm_attack, deepfool_attack])
adv_trainer.fit(x_test, y_test, nb_epochs=10)

# Re-evaluar el modelo defensivo en ejemplos adversariales generados por FGSM
x_test_adv_fgsm = fgsm_attack.generate(x=x_test)
y_pred_adv_fgsm_defended = classifier.predict(x_test_adv_fgsm)
y_pred_adv_fgsm_defended_indices = np.argmax(y_pred_adv_fgsm_defended, axis=1)
y_true_indices = np.argmax(y_test, axis=1)
accuracy_fgsm_defended = np.mean(y_pred_adv_fgsm_defended_indices == y_true_indices)
print("Defended Model - FGSM Attack - Accuracy:", accuracy_fgsm_defended)

# Re-evaluar el modelo defensivo en ejemplos adversariales generados por DeepFool
x_test_adv_deepfool = deepfool_attack.generate(x=x_test)
y_pred_adv_deepfool_defended = logits_classifier.predict(x_test_adv_deepfool)
y_pred_adv_deepfool_defended_indices = np.argmax(y_pred_adv_deepfool_defended, axis=1)
accuracy_deepfool_defended = np.mean(y_pred_adv_deepfool_defended_indices == y_true_indices)
print("Defended Model - DeepFool Attack - Accuracy:", accuracy_deepfool_defended)


Found 9339 images belonging to 25 classes.


Precompute adv samples:   0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: 